## Load dataset

nspi https://www.kaggle.com/code/ashkhagan/audio-signal-processing-librosa#Visualizing-Audio

## Preprocessing

Il est généralement recommandé de normaliser les données avant l'EDA (Exploratory Data Analysis), car cela peut aider à mieux comprendre la distribution des données et faciliter l'interprétation des résultats de l'analyse exploratoire.

La normalisation des données implique de transformer les données pour qu'elles aient une moyenne nulle et une variance unitaire ou dans une plage spécifique. Cette transformation peut aider à éliminer les biais dans les données et à faciliter la comparaison entre différentes caractéristiques.

Lorsque les données ne sont pas normalisées, il peut être difficile de déterminer si les différences entre les caractéristiques sont dues à des différences réelles entre les données ou simplement à des différences d'échelle. En normalisant les données, on peut s'assurer que les différences observées sont dues à des différences réelles entre les données et non à des différences d'échelle.

Cependant, il est important de noter que la normalisation des données doit être effectuée avec précaution, en fonction du type de données et de l'objectif de l'analyse. Par exemple, pour les données audio, il peut être nécessaire de normaliser les caractéristiques telles que l'amplitude et la fréquence, mais il peut être préférable de ne pas normaliser les caractéristiques liées à la dynamique du son.

## EDA

Mieux comprendre les données audio, d'identifier les éventuels problèmes et d'optimiser la sélection des caractéristiques audio pour construire un modèle de classification d'audio performant.

In [ ]:
dataset.info()

In [ ]:
dataset.describe().T

### Class distribution

In [ ]:
dataset['class'].value_counts()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
dataset['class'].value_counts().plot(kind='barh', ax=ax1)
dataset['class'].value_counts().plot(kind='pie', ax=ax2, autopct='%.0f%%')

### Visualize audio (time, freq domains) by class

In [ ]:
class_names = dataset['class'].unique().tolist()
print(class_names)

In [ ]:
def plot_audio_signals(audio_files, n_fft=2048, hop_length=512, figsize=(20, 8), title=None):
    """
    Affiche les tracés du domaine temporel et du domaine fréquentiel d'une liste d'audios.

    Args:
        audio_files (list): Liste des chemins des fichiers audio à visualiser (maximum 5).
        n_fft (int): Taille de la fenêtre de transformation de Fourier.
        hop_length (int): Pas de la fenêtre de transformation de Fourier.
        figsize (tuple): Taille de la figure (largeur, hauteur).
        title (str): Titre de la figure.
    """
    if len(audio_files) > 5:
        raise ValueError("Le nombre maximum d'audios pris en charge est de 5.")

    fig, axes = plt.subplots(nrows=2, ncols=len(audio_files), figsize=figsize)
    cbar_ax = fig.add_axes([0.925, 0.1, .02, .775]) # [left, bottom, width, height]

    for i, audio_file in enumerate(audio_files):
        # Chargement du fichier audio
        y, sr = librosa.load(audio_file)
        audio_name = os.path.basename(audio_file)
        # Tracé du domaine temporel
        librosa.display.waveshow(y, alpha=0.5, ax=axes[0, i])
        axes[0, i].set_title(f"{audio_name}", fontsize=10)
        axes[0, i].set_ylabel("Amplitude")
        axes[0, i].set_xlabel("Time (s)", fontsize=10)

        # Tracé du domaine fréquentiel
        D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
        S_db = librosa.amplitude_to_db(abs(D), ref=np.max)
        img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', ax=axes[1, i])
        axes[1, i].set_title(f"{audio_name}", fontsize=10)
        axes[1, i].set_xlabel("Time (s)", fontsize=10)
        axes[1, i].set_ylabel("Frequency (Hz)")
        #axes[1, i].set_yticks([100, 200, 500, 1000, 2000, 5000])
        #axes[1, i].set_yticklabels(['100', '200', '500', '1k', '2k', '5k'])

    # Ajout de la barre de couleur
    plt.colorbar(img, format="%+2.0f dB", ax=axes.ravel().tolist(), cax=cbar_ax)
    plt.subplots_adjust(wspace=0.4, hspace=0.3)
    if title:
        plt.suptitle(title, fontsize=16)
    plt.show()

for class_name in class_names:
    # get all file_path for this class
    file_paths = dataset[dataset['class'] == class_name]['file_path'].tolist()
    random.shuffle(file_paths)
    plot_audio_signals(file_paths[:5], title=class_name)

In [ ]:
dataset.boxplot()

## Correlation

Cette étape consiste à explorer les relations entre les caractéristiques audio en calculant les corrélations entre elles. Cela permet de mieux comprendre les relations entre les différentes caractéristiques et de détecter les